In [1]:
import datetime
import os

import numpy  as np
import pandas as pd

In [2]:
# Variables

pickleDir = '../Data/Pickle'
inputFile = '../Data/Fields/D11.csv'


In [3]:
print('input datas:',datetime.datetime.now())

frame = pd.read_csv(inputFile)

print('end input',datetime.datetime.now())
print(frame.tail())

input datas: 2016-05-26 00:00:01.243289
end input 2016-05-26 00:00:13.670753
          user_id  is_booking  dt0m  dt0y  hotel_cluster  countCnt  count
15015383  1198784           0     4  2015             26         2      1
15015384  1198784           0     4  2015             68         6      1
15015385  1198784           0     4  2015             95         1      1
15015386  1198784           0     8  2014             79         1      1
15015387  1198784           1     8  2014             79         1      1


In [4]:
grouped=frame.groupby(['user_id','dt0m','is_booking']); print('user+dt0m+book =',len(grouped))
print('grouped datas:',datetime.datetime.now())

user+dt0m+book = 4379395
grouped datas: 2016-05-26 00:10:05.453479


In [5]:
print(len(grouped))

4379395


In [6]:
i        = -1
nLargest =  5; n = nLargest;

dcol = ['count','countCnt'] + ['n'+str(i) for i in range(5)] + ['cc'+str(i) for i in range(100)]
dcol = ['count','countCnt'] + ['n'+str(i) for i in range(n)]  + ['v'+str(i) for i in range(n)] 
def f (xgroup) :
    global i,n
    d00  = np.zeros((100));d00[xgroup['hotel_cluster']]+=xgroup['count']
    dsum = xgroup['countCnt'].sum()
    dcnt = xgroup['count'].sum()
    d05  = pd.Series(d00); 
    n05=d05[d05>0].nlargest(n).index.tolist()+[-1]*n; n05=n05[0:n];
    v05=d05[d05>0].nlargest(n).tolist()+[-1]*n;       v05=v05[0:n];
    ddd  = [dcnt,dsum] + n05 + v05 # + d00.tolist()
    #i   += 1
    #if (i%10000==0) : print('5 :',datetime.datetime.now()," i = ",i)

    
    return(pd.DataFrame(ddd,index=dcol).T)



In [37]:

def ff (transx,nn,filepickle) :
    trans0 = transx.query('is_booking==0')[['user_id','dt0m','is_booking','count','countCnt']+ \
                                           ['n'+str(i) for i in range(nn)]+['v'+str(i) for i in range(nn)]]
    trans0.index = [trans0.user_id.as_matrix(),trans0.dt0m.as_matrix()]

    trans1 = transx.query('is_booking==1')[['user_id','dt0m','is_booking','count','countCnt']+ \
                                           ['n'+str(i) for i in range(nn)]+['v'+str(i) for i in range(nn)]]
    trans1.index = [trans1.user_id.as_matrix(),trans1.dt0m.as_matrix()]

    trans01 = pd.concat([trans0,trans1],axis=1)

    trans01.columns = ['user_id','dt0m','bo0','count0','countCnt0']+['b0n'+str(i) for i in range(nn)]+['b0v'+str(i) for i in range(nn)]+ \
                        ['h1','dt01','bo1','count1','countCnt1']+['b1n'+str(i) for i in range(nn)]+['b1v'+str(i) for i in range(nn)]
    trans01 = trans01[['user_id','dt0m','count0','countCnt0','count1','countCnt1'] + \
                      ['b0n'+str(i) for i in range(nn)]+['b1n'+str(i) for i in range(nn)]+ \
                      ['b0v'+str(i) for i in range(nn)]+['b1v'+str(i) for i in range(nn)]]
    
    trans01.to_pickle(filepickle)
    
    return(trans01)


In [40]:
print(len(grouped))
print(frame['user_id'].min(),frame['user_id'].max())

ll = frame['user_id'].max()+1
l = [i for i in range(0,ll,17000)]
if (l[len(l)-1]!=ll) : l.append(ll)
mm = [(l[i],l[i+1]) for i in range(len(l)-1)]
#print(m)
#for i1,i2 in mm[-10:] : print((i2-i1))
print(len(mm),mm[0:5])
print(len(mm),mm[-10:])
print(nLargest,n)

4379395
0 1198784
71 [(0, 17000), (17000, 34000), (34000, 51000), (51000, 68000), (68000, 85000)]
71 [(1037000, 1054000), (1054000, 1071000), (1071000, 1088000), (1088000, 1105000), (1105000, 1122000), (1122000, 1139000), (1139000, 1156000), (1156000, 1173000), (1173000, 1190000), (1190000, 1198785)]
5 5


In [41]:
fres = []
i1,i2 = 10000,20000
for i1,i2 in mm :
    print('4 :',datetime.datetime.now(),' i1,i2=',i1,i2)
    cond = '('+str(i1)+'<=user_id)&(user_id<'+str(i2)+')'
    grouped100 = frame.query(cond).groupby(['user_id','dt0m','is_booking']); 
    #print('user+book =',len(grouped100))
    #print('grouped (x in) datas:',datetime.datetime.now())
    i, n = -1, nLargest
    trans   = grouped100.apply(f);
    trans   = trans.reset_index()
    ffile   = os.path.join(pickleDir,'trans01-'+str(i1+1000000000)+'.pck')
    trans01 = ff(trans,n,ffile)
    fres.append(ffile)
    #print('6 :',datetime.datetime.now())
    
print('7 :',datetime.datetime.now())
print('fres=',len(fres))
trans01.head()

4 : 2016-05-26 01:39:50.694792  i1,i2= 0 17000


C:\Anaconda3-64\lib\site-packages\ipykernel\__main__.py:8: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


4 : 2016-05-26 01:43:39.586588  i1,i2= 17000 34000
4 : 2016-05-26 01:48:59.329848  i1,i2= 34000 51000
4 : 2016-05-26 01:54:49.551550  i1,i2= 51000 68000
4 : 2016-05-26 02:00:59.931143  i1,i2= 68000 85000
4 : 2016-05-26 02:08:58.095166  i1,i2= 85000 102000
4 : 2016-05-26 02:15:28.019238  i1,i2= 102000 119000
4 : 2016-05-26 02:21:47.772370  i1,i2= 119000 136000
4 : 2016-05-26 02:28:23.576804  i1,i2= 136000 153000
4 : 2016-05-26 02:34:57.082974  i1,i2= 153000 170000
4 : 2016-05-26 02:42:20.701682  i1,i2= 170000 187000
4 : 2016-05-26 02:48:56.414023  i1,i2= 187000 204000
4 : 2016-05-26 02:56:17.235902  i1,i2= 204000 221000
4 : 2016-05-26 03:01:17.314362  i1,i2= 221000 238000
4 : 2016-05-26 03:05:53.347965  i1,i2= 238000 255000
4 : 2016-05-26 03:14:20.737088  i1,i2= 255000 272000
4 : 2016-05-26 03:26:04.257042  i1,i2= 272000 289000
4 : 2016-05-26 03:34:18.938004  i1,i2= 289000 306000
4 : 2016-05-26 04:05:08.829038  i1,i2= 306000 323000
4 : 2016-05-26 04:16:26.091821  i1,i2= 323000 340000
4 

user_id  dt0m  count0  countCnt0  count1  countCnt1  b0n0  b0n1  \
1190000 1   1190000.0   1.0    55.0      104.0     NaN        NaN   4.0   1.0   
1190001 1   1190001.0   1.0     3.0        4.0     1.0        2.0   9.0  42.0   
        7   1190001.0   7.0     2.0        2.0     NaN        NaN  41.0  -1.0   
1190003 10  1190003.0  10.0     2.0        2.0     1.0        1.0  25.0  55.0   
1190005 1   1190005.0   1.0     7.0        7.0     1.0        1.0   7.0  10.0   

            b0n2  b0n3  ...   b0v0  b0v1  b0v2  b0v3  b0v4  b1v0  b1v1  b1v2  \
1190000 1   19.0  55.0  ...   13.0  11.0   5.0   5.0   3.0   NaN   NaN   NaN   
1190001 1   -1.0  -1.0  ...    2.0   1.0  -1.0  -1.0  -1.0   1.0  -1.0  -1.0   
        7   -1.0  -1.0  ...    2.0  -1.0  -1.0  -1.0  -1.0   NaN   NaN   NaN   
1190003 10  -1.0  -1.0  ...    1.0   1.0  -1.0  -1.0  -1.0   1.0  -1.0  -1.0   
1190005 1   12.0  14.0  ...    1.0   1.0   1.0   1.0   1.0   1.0  -1.0  -1.0   

            b1v3  b1v4  
1190000 1    NaN   NaN  
1190001 1   -1.0  -1.0  
        7    NaN   NaN  
1190003 10  -1.0  -1.0  
1190005 1   -1.0  -1.0  

[5 rows x 26 columns]

In [42]:
ff  = pd.read_pickle(fres[0])
for ifile in fres[1:len(fres)] :
    ff1 = pd.read_pickle(ifile)
    ff  = pd.concat([ff,ff1])
    del ff1
print(ff.shape)
ff.tail()

(3120154, 26)


user_id  dt0m  count0  countCnt0  count1  countCnt1  b0n0  b0n1  \
1198783 8   1198783.0   8.0     7.0       12.0     1.0        1.0  91.0  94.0   
        9   1198783.0   9.0     1.0        1.0     1.0        1.0  47.0  -1.0   
        12  1198783.0  12.0    12.0       16.0     NaN        NaN  23.0  39.0   
1198784 4   1198784.0   4.0     3.0        9.0     NaN        NaN  26.0  68.0   
        8   1198784.0   8.0     1.0        1.0     1.0        1.0  79.0  -1.0   

            b0n2  b0n3  ...   b0v0  b0v1  b0v2  b0v3  b0v4  b1v0  b1v1  b1v2  \
1198783 8   15.0  47.0  ...    2.0   2.0   1.0   1.0   1.0   1.0  -1.0  -1.0   
        9   -1.0  -1.0  ...    1.0  -1.0  -1.0  -1.0  -1.0   1.0  -1.0  -1.0   
        12   5.0   6.0  ...    2.0   2.0   1.0   1.0   1.0   NaN   NaN   NaN   
1198784 4   95.0  -1.0  ...    1.0   1.0   1.0  -1.0  -1.0   NaN   NaN   NaN   
        8   -1.0  -1.0  ...    1.0  -1.0  -1.0  -1.0  -1.0   1.0  -1.0  -1.0   

            b1v3  b1v4  
1198783 8   -1.0  -1.0  
        9   -1.0  -1.0  
        12   NaN   NaN  
1198784 4    NaN   NaN  
        8   -1.0  -1.0  

[5 rows x 26 columns]

In [43]:
print(ff.shape)
ff.to_pickle(os.path.join(pickleDir,'trans01-user_id-dt0m-all.pck'))
del ff

(3120154, 26)


In [44]:
ff = pd.read_pickle(os.path.join(pickleDir,'trans01-user_id-dt0m-all.pck'))
ff.shape

(3120154, 26)

In [47]:
print('result(len,0+1,only 0) -->',ff.shape[0],ff[pd.notnull(ff.count1)].shape[0],ff[pd.isnull(ff.count1)].shape[0])
ff[pd.notnull(ff.count1)].shape[0]/ff.shape[0]

result(len,0+1,only 0) --> 3120154 1276239 1843915


0.40903077219906453

In [49]:
ff[ff.b0n0==ff.b1n0].shape[0]/ff.shape[0]

0.23784531148142046

In [50]:
print('count0 = (mean,max)',ff['count0'].mean(),ff['count0'].max())
print('count1 = (mean,max)',ff['count1'].mean(),ff['count1'].max())

count0 = (mean,max) 7.141045438901557 491.0
count1 = (mean,max) 1.5557540554708014 56.0


In [159]:
import matplotlib
matplotlib.style.use('ggplot')

In [160]:
ff[['count0','count1']].plot(x='count0',y='count1')

In [52]:
ff.head()

user_id  dt0m  count0  countCnt0  count1  countCnt1  b0n0  b0n1  b0n2  \
0 9       0.0   9.0     1.0        1.0     NaN        NaN  19.0  -1.0  -1.0   
  12      0.0  12.0     5.0        6.0     NaN        NaN  28.0  40.0  56.0   
1 11      1.0  11.0     4.0        4.0     NaN        NaN  20.0  60.0  -1.0   
3 2       3.0   2.0     5.0        5.0     NaN        NaN  94.0   5.0  47.0   
  10      3.0  10.0     3.0        3.0     NaN        NaN  18.0   2.0  -1.0   

      b0n3  ...   b0v0  b0v1  b0v2  b0v3  b0v4  b1v0  b1v1  b1v2  b1v3  b1v4  
0 9   -1.0  ...    1.0  -1.0  -1.0  -1.0  -1.0   NaN   NaN   NaN   NaN   NaN  
  12  72.0  ...    1.0   1.0   1.0   1.0   1.0   NaN   NaN   NaN   NaN   NaN  
1 11  -1.0  ...    3.0   1.0  -1.0  -1.0  -1.0   NaN   NaN   NaN   NaN   NaN  
3 2   48.0  ...    2.0   1.0   1.0   1.0  -1.0   NaN   NaN   NaN   NaN   NaN  
  10  -1.0  ...    2.0   1.0  -1.0  -1.0  -1.0   NaN   NaN   NaN   NaN   NaN  

[5 rows x 26 columns]